In [23]:
using Pkg
using Revise
Pkg.activate("..") 
Pkg.instantiate()
using JPEC, Plots
gr() 

  Activating project at `~/git/JPEC11/JPEC`


Plots.GRBackend()

In [24]:
using Revise
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

### Reading Eq

In [25]:
# 1. Define the input parameters for the equilibrium solver.
#    - eq_filename: The name of the g-file we just created.
#    - eq_type: "efit" for a standard g-file.
#    - jac_type: "boozer" or "hamada" for the output coordinates.
#    - mpsi, mtheta: Resolution of the output grid.
    equil_control = JPEC.Equilibrium.EquilibriumControl(;
    eq_filename="beta_1.00",        # eq_filename
    eq_type="efit",          # eq_type
    jac_type="hamada",        # jac_type
    grid_type="ldp",
    psilow=0.01,             # psilow
    psihigh=0.990)             # psihigh
equil_config = JPEC.Equilibrium.EquilibriumConfig(equil_control,JPEC.Equilibrium.EquilibriumOutput())
# 2. Run the main equilibrium setup function.
#    This will read the file, solve the direct problem, and return the final object.
println("Starting equilibrium reconstruction...")
plasma_eq = JPEC.Equilibrium.setup_equilibrium(equil_config)
#plasma_eq = JPEC.Equilibrium.setup_equilibrium("./DIIID_example/equil.toml")
println("Equilibrium reconstruction complete.")

┌ Info: Forcing hamada coordinate jacobian exponents: power_*
└ @ JPEC.Equilibrium /Users/seoda-eun/git/JPEC11/JPEC/src/Equilibrium/EquilibriumTypes.jl:48


Starting equilibrium reconstruction...
Equilibrium file: beta_1.00
--> Processing EFIT g-file: beta_1.00
--> Parsed from header: nw=129, nh=128
--> All main data blocks parsed successfully.
--> Creating 1D profile splines...
--> 1D Spline fitting complete.
--> Creating 2D psi spline...
--> 2D Spline fitting complete.
--- Starting Direct Equilibrium Processing ---
Finding magnetic axis...
  Iter  1: R = 3.115238, Z = 0.000008, |ΔR|=1.15e-01, |ΔZ|=1.11e-05
  Iter  2: R = 3.117635, Z = -0.000003, |ΔR|=2.40e-03, |ΔZ|=1.10e-05
  Iter  3: R = 3.117636, Z = -0.000003, |ΔR|=1.07e-06, |ΔZ|=1.04e-09
  Iter  4: R = 3.117636, Z = -0.000003, |ΔR|=2.12e-13, |ΔZ|=5.62e-16
Magnetic axis found at R=3.1176357129137466, Z=-2.9184696943092166e-6.
Finding inboard separatrix crossing...
  Restart attempt 1/6 with initial R = 2.675802
inboard separatrix found at R=1.5000175239830411.
Finding outboard separatrix crossing...
  Restart attempt 1/6 with initial R = 3.500684
outboard separatrix found at R=4.49999

In [26]:
fieldnames(typeof(plasma_eq.rzphi))

(:handle, :_xs, :_ys, :_fs, :mx, :my, :nqty, :bctypex, :bctypey, :_fsx, :_fsy, :_fsxy)

### CAlc. Metric

In [27]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

In [28]:
# Fourfit Metric Calculation Example
println("📚 Fourfit Metric Calculation")
println("="^30)

using .FourfitMetric
metric_result = nothing
try
    # Metric tensor calculation
    metric_result = FourfitMetric.fourfit_make_metric(
        plasma_eq.rzphi,  # 2D geometry spline
        plasma_eq.sq;     # 1D profile spline
        mpsi=128,          # Radial grid points
        mtheta=256,        # Poloidal grid points  
        mband=10,          # Fourier bandwidth
        fft_flag=true,    # Use FFT
        verbose=true      # Detailed output
    )
    
    println("✅ Metric calculation complete!")
    println("   Grid size: $(size(metric_result.fs))")
 
catch e
    println("❌Error: $e")
end

📚 Fourfit Metric Calculation
🔧 Starting metric tensor calculation...


   Grid: 129 × 257, mband: 10
📊 Computing metric tensor components on grid...
---- DEBUG: ipsi=64.0, itheta=128.0 ----
psi_coord = 0.4999999999999999
theta_coord = 0.49609375
bicube_eval: f = [1.0556705372876192, -0.0033389596882782975, -0.02808040164119698, 186.5043062876643]
bicube_eval: fx = [2.4382568339151183, -0.0033018151911684934, -0.06546817349115953, 107.74688881364901]
bicube_eval: fy = [-0.7733913169930531, 0.852669494134877, 7.186190745690982, 0.0]
rfac = 1.0274582898043205
eta = 3.0960696585289678
r = 2.091236149391108
jac = 186.5043062876643
jac1 = 107.74688881364901
Contravariant basis vectors v:
  v[1,:] = [0.006362039923106128, -0.00011428992988609545, -0.0007340817687509711]
  v[2,:] = [-0.0020179770918524655, 0.06412880622547774, 0.08057734517201763]
  v[3,:] = [0.0, 0.0, 0.07045212257689355]
g11 = 0.007651803601175929
g22 = 1.9786772597603564
g33 = 0.9257144181130735
g23 = 1.0587560384372956
g31 = -0.00964555860847258
g12 = -0.014793164914085334
jac = 186.504306287

In [29]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

### Calc. Matric

In [30]:
using Revise

In [31]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

In [32]:
# Fourfit Matrix Calculation - Complete Example
println("🚀 Fourfit Matrix Calculation")
println("="^40)

using .FourfitMetric
matrix_result = nothing
try
    # Step 1: Matrix calculation with correct arguments
    println("2️⃣  Calculating MHD Coefficient Matrix...")
    
    matrix_result = FourfitMetric.fourfit_make_matrix(
        metric_result,       # metric tensor data
        plasma_eq.sq,        # 1D profile spline
        plasma_eq.rzphi,     # 2D geometry spline
        1.367;               # psio (toroidal flux normalization)
        nn=1,                # toroidal mode number
        mlow=-4,             # minimum poloidal mode
        mhigh=4,             # maximum poloidal mode
        power_flag=false,
        feval_flag=false,
        sas_flag=true,
        verbose=true
    )
    
    # Step 2: Analyze results
    println("3️⃣  Analyzing Results:")
    println("   Matrix storage sizes:")
    println("      F-matrix: $(size(matrix_result.fmats))")
    println("      G-matrix: $(size(matrix_result.gmats))")
    println("      K-matrix: $(size(matrix_result.kmats))")
    println("   Mode range: $(matrix_result.mlow) to $(matrix_result.mhigh)")
    println("   Number of modes: $(matrix_result.mpert)")
    
    println("✅ Matrix calculation completed successfully!")
    
catch e
    println("❌ Error occurred: $e")
    println("🔍 Error type: $(typeof(e))")
end

🚀 Fourfit Matrix Calculation
2️⃣  Calculating MHD Coefficient Matrix...
🚀 Fourfit Matrix Calculation - Complete Implementation


📊 Configuration:
   Mode range: -4 to 4 (total: 9 modes)
   Toroidal mode: n = 1
   Psi surfaces: 0 to 128
   Metric data size: (129, 88)

1️⃣  Allocating matrix storage...
   ✅ Matrix dimensions: 9 × 9
   ✅ Hermitian storage size: 44
   ✅ K-matrix storage size: 189

2️⃣  Computing matrices on each flux surface...
   Progress: 0.0% (ipsi = 0/128, ψ = 0.01)
   Progress: 9.4% (ipsi = 12/128, ψ = 0.0311)
   Progress: 18.8% (ipsi = 24/128, ψ = 0.0926)
   Progress: 28.1% (ipsi = 36/128, ψ = 0.1891)
   Progress: 37.5% (ipsi = 48/128, ψ = 0.3125)
   Progress: 46.9% (ipsi = 60/128, ψ = 0.452)
---- DEBUG: ipsi=64, dmat calculation ----
  ipert = 1, jpert = 1, m1 = -4, m2 = -4, dm = 0
  chi1 = 8.589114314914495, twopi = 6.283185307179586
  dm_idx = 11
  g23[11] = -0.5322619737134378 + 0.0im
  g33[11] = 2.1189718283137484 + 0.0im
  m1 = -4, nn = 1
  dmat[1, 1] = -486.14276433068807 + 0.0im
  최종 계산식: dmat[1, 1] = twopi * chi1 * (g23[11] + g33[11] * m1 / nn)
---- DEBUG: ipsi=64, dmat calculation --

### Redaing imats(original DCON output)

### comparison

In [33]:
matrix_result.mband

10

In [34]:
reshape(matrix_result.cmats[129,:], 9, 9)

9×9 Matrix{ComplexF64}:
 -160.518+8381.35im   48.4295+25370.8im  …    0.0577155-3.45449im
  43.0037-13249.9im  -106.659+6889.28im     -0.00678479-6.22431im
  12.6718-11951.4im   27.0871-7945.49im      -0.0715736-10.2844im
  41.3233-3293.74im  -6.54038-8632.63im        0.229747+12.2927im
 -53.4983-1172.82im   51.1224-2237.51im       -0.553568+77.3936im
  26.7393-25.3015im   -52.771-825.558im  …     0.786456+271.421im
 -4.20338-59.0536im   25.8943+22.7653im       -0.702907+513.794im
 -12.4659+46.1599im  -3.02029-48.5392im       -0.305998+681.089im
  24.8495-18.5606im  -12.6389+34.3747im         1.57153-3555.23im

In [35]:
reader.C[end, :, :]

9×9 Matrix{ComplexF64}:
 -652.336+25548.3im   986.022-40032.0im  …  -128.962-135.822im
  588.864+76968.2im  -898.582+21002.3im      195.501+170.465im
 -551.077+45727.9im   744.083+56671.3im     -185.408-127.682im
  507.956+13528.9im  -593.193+34326.8im      305.237+185.816im
 -203.472+3807.26im   455.896+9839.28im     -457.467-352.287im
   165.78+376.755im  -177.578+2635.88im  …    596.45+145.62im
 -94.1679-2.81625im   113.993+228.474im     -625.911-207.233im
 0.771125-66.8254im  -47.4386-17.1982im      432.012+4986.41im
 -1.94857-10.8309im  0.300178-19.7919im     -41.7906-10819.8im

In [36]:
(reshape(matrix_result.cmats[64, :], 9, 9)./reader.C[64,:,:])

9×9 Matrix{ComplexF64}:
  0.998996-0.00140349im    -1.35714+0.000605056im  …  0.162482-0.230188im
 -0.736224-0.000337463im   0.999011-0.000816849im     0.171071-0.236911im
  -1.08705+0.00301865im   -0.638248-0.000212444im     0.714771-0.330749im
  -3.00369+0.0645997im     -1.16987+0.0023921im        1.13085-0.259181im
  -3.94706+0.365053im       -4.2875+0.0499779im        2.04815-0.301886im
  -12.8191+0.972184im      -14.0045+0.966563im     …   8.28046-0.306928im
   7.28384-0.515753im       6.50119-0.204228im         3.90476+0.104696im
   1.15659-1.79977im        2.34717-0.0354796im       -0.90097+0.00101946im
   1.98205+0.12616im       0.935381-0.957591im        0.998884+0.00224595im

In [37]:
reshape(matrix_result.cmats[64, :], 9, 9)

9×9 Matrix{ComplexF64}:
 -0.00519793+663.83im    -0.00019697+1779.9im    …     0.0012721-0.0896604im
 -0.00258375-1311.1im    -0.00126157+513.963im       -0.00823695+0.247327im
   0.0226912-316.058im   -0.00365526-693.471im       -0.00585143-0.723858im
 -0.00359159+35.2538im     0.0132431-208.808im        0.00663869+2.1805im
  0.00141957-11.7767im   -0.00485994+30.0941im      -0.000679061-5.94239im
 0.000512998+4.11135im    0.00578304-9.52785im   …     0.0042107+19.5645im
  -0.0012122-1.45498im    0.00588839+3.24188im        0.00138987-44.0674im
  -0.0111881+0.508427im   -0.0035625-1.18409im        0.00699739+600.754im
 -0.00390826-0.165324im   -0.0128069+0.415238im        -0.012071-535.107im

In [38]:
reader.C[64,:,:]

9×9 Matrix{ComplexF64}:
   -0.938752+664.496im      0.584862-1311.51im   …    0.262581-0.179821im
    0.819795+1780.84im     -0.421924+514.471im       -0.702682+0.47263im
   -0.828248+290.745im      0.367382+1086.52im        0.379229-0.837232im
      0.2535-11.7314im     -0.376284+178.488im       -0.414294+1.83325im
   -0.273966+2.95832im     0.0829406-7.01807im        0.418225-2.8397im
   0.0241443-0.31889im    -0.0471445+0.67709im   …  -0.0869502+2.3595im
    0.013908-0.198769im   -0.0147446+0.498196im      -0.302021-11.2775im
   -0.202757+0.124082im   0.00610644-0.504383im       0.746713-666.785im
 -0.00725167-0.0829494im   -0.228586+0.20991im        -1.21659-535.702im

In [40]:
reader.C[64,:,:]
sq=plasma_eq.sq
ipsi =64
psifac = sq.xs[ipsi + 1]

f_vals, f1_vals = JPEC.SplinesMod.spline_eval(sq, [psifac], 1)
f_vals[1,:][4]
f1_vals[1,:][2]

-0.03673591207562535

In [41]:
using CSV, DataFrames

mutable struct ImatsReaderCSV
    psi       :: Vector{Float64}       # ψ value list
    m_values  :: Vector{Int}           # m value list
    mpsi      :: Int                   # number of ψ values
    mpert     :: Int                   # number of modes
    A         :: Array{ComplexF64,3}   # matrix A (ψ x mpert x mpert)
    B         :: Array{ComplexF64,3}   # matrix B
    C         :: Array{ComplexF64,3}   # matrix C
    D         :: Array{ComplexF64,3}   # matrix D
    E         :: Array{ComplexF64,3}   # matrix E
    H         :: Array{ComplexF64,3}   # matrix H
end

function ImatsReaderCSV(path::AbstractString)
    # CSV 파일을 헤더는 6번째 줄부터 읽고, 공백을 구분자로 사용
    df = CSV.read(path, DataFrame; skipto=6, delim=' ', ignorerepeated=true)
    println("Found columns: ", names(df))
    
    # 실제 헤더에 맞게 컬럼 이름 지정
    # 컬럼: "psi", "m1", "m2", "real(Ai)", "imag(Ai)", "real(Bi)", "imag(Bi)", 
    #      "real(Ci)", "imag(Ci)", "real(Di)", "imag(Di)", "real(Ei)", "imag(Ei)", 
    #      "real(Hi)", "imag(Hi)"
    psi_vals = sort(unique(df[!, "psi"]))
    m_vals = sort(unique(vcat(df[!, "m1"], df[!, "m2"])))
    mpsi = length(psi_vals)
    mpert = length(m_vals)
    shp = (mpsi, mpert, mpert)
    
    # 빈 행렬 생성
    A = zeros(ComplexF64, shp)
    B = zeros(ComplexF64, shp)
    C = zeros(ComplexF64, shp)
    D = zeros(ComplexF64, shp)
    E = zeros(ComplexF64, shp)
    H = zeros(ComplexF64, shp)
    
    # 값 → 인덱스 변환 사전 생성
    psi_idx = Dict(v => i for (i, v) in enumerate(psi_vals))
    m_idx = Dict(v => i for (i, v) in enumerate(m_vals))
    
    # 각 행(row)마다, 해당 (ψ, m1, m2) 인덱스를 찾아 값을 저장
    for row in eachrow(df)
        k = psi_idx[row["psi"]]
        i = m_idx[row["m1"]]
        j = m_idx[row["m2"]]
        A[k, i, j] = ComplexF64(row["real(Ai)"], row["imag(Ai)"])
        B[k, i, j] = ComplexF64(row["real(Bi)"], row["imag(Bi)"])
        C[k, i, j] = ComplexF64(row["real(Ci)"], row["imag(Ci)"])
        D[k, i, j] = ComplexF64(row["real(Di)"], row["imag(Di)"])
        E[k, i, j] = ComplexF64(row["real(Ei)"], row["imag(Ei)"])
        H[k, i, j] = ComplexF64(row["real(Hi)"], row["imag(Hi)"])
    end
    
    return ImatsReaderCSV(psi_vals, m_vals, mpsi, mpert, A, B, C, D, E, H)
end

ImatsReaderCSV

In [42]:
reader=ImatsReaderCSV("../src/DCON/validate/imats.out")

Found columns: ["psi", "m1", "m2", "real(Ai)", "imag(Ai)", "real(Bi)", "imag(Bi)", "real(Ci)", "imag(Ci)", "real(Di)", "imag(Di)", "real(Ei)", "imag(Ei)", "real(Hi)", "imag(Hi)"]


ImatsReaderCSV([0.00999999978, 0.010147579, 0.0105902264, 0.0113276765, 0.0123594841, 0.0136850281, 0.0153035102, 0.0172139555, 0.0194152128, 0.0219059568  …  0.978094041, 0.9805848, 0.982786059, 0.984696507, 0.986314952, 0.9876405, 0.988672316, 0.989409745, 0.989852428, 0.99000001], [-4, -3, -2, -1, 0, 1, 2, 3, 4], 129, 9, ComplexF64[0.0 + 0.0im 79.5266571 - 0.000821601774im … -0.000204464101 - 6.27206609e-5im -0.00156646129 - 9.75607254e-5im; 1530.38721 + 0.0im 80.1111069 - 0.000770779676im … -0.00023349328 - 7.30999091e-5im -0.00164888229 - 9.55864016e-5im; … ; 1268.09229 + 0.0im 781.701111 - 0.0895105377im … -3.04324365 + 0.0261531398im 1.90980232 - 0.0214035008im; 1267.49536 + 0.0im 782.051941 - 0.0595168881im … -3.06308794 + 0.0245061945im 1.91602159 - 0.00606359495im;;; 0.0 + 0.0im 861.666626 + 0.0im … -0.000896688027 - 0.000150625565im -0.00057099195 + 3.65024534e-5im; 80.1111069 + 0.000770779676im 861.703186 + 0.0im … -0.000932913856 - 0.000148287581im -0.000606337679 + 2.9944

In [43]:
@show (reshape(matrix_result.cmats[120, :], 9, 9)./reader.C[120,:,:])

reshape(matrix_result.cmats[120, :], 9, 9) ./ reader.C[120, :, :] = ComplexF64[0.9659964688581583 - 0.003906447518452728im -1.8047493660492817 - 0.0011075314181223756im -1.1929371964177793 + 0.0013315538953356779im -1.3115124738017303 + 0.0002535922159699826im -0.9776011694757448 + 0.003919214218071369im 1.2794737876343985 + 0.09303064260687373im 0.05654007824588269 + 0.007637623737816333im -0.20920746064138748 - 0.012199288433851433im 0.060277984263243184 + 0.004208387832230128im; -0.5164850520032817 + 0.00025252133086524597im 0.9658987100203434 - 0.003911171893013623im -2.210148026083143 - 0.0024751852872289936im -1.2340640902386106 + 0.001383332375247552im -1.4146605281687068 + 0.0007450601341510041im -0.9516111337114007 + 0.0034888214086415846im 0.41053898328086286 + 0.01307612213965991im 0.1136827026818136 + 0.007549676587007126im -0.02179040502212196 - 0.0026018508615551286im; -0.7797018043801948 - 0.0012176325215894571im -0.4214906258169023 + 0.0004859596315045892im 0.9657427601

9×9 Matrix{ComplexF64}:
  0.965996-0.00390645im    -1.80475-0.00110753im  …    0.060278+0.00420839im
 -0.516485+0.000252521im   0.965899-0.00391117im     -0.0217904-0.00260185im
 -0.779702-0.00121763im   -0.421491+0.00048596im       0.204622+0.0097515im
 -0.704758+0.00108699im   -0.753456-0.00135692im       0.030021-0.000273124im
 -0.944267-0.00607751im   -0.651937+0.00133163im      -0.135931-0.00157416im
   0.76045+0.0591982im     -0.97172-0.0069406im   …     1.08429+0.00871119im
   13.5497-0.509293im       2.31119+0.159209im         -4.40106-0.137498im
  -4.22468-0.0180267im      7.90568+0.0117201im        0.715293-0.00149739im
   13.1263-0.0551291im     -36.8625+7.7337im           0.967445-0.000143525im

In [46]:
@show (reshape(matrix_result.dmat, 9, 9)./reader.D[64,:,:])

reshape(matrix_result.dmat, 9, 9) ./ reader.D[64, :, :] = ComplexF64[-1.0871468595023772 + 0.0im 2.451301612679235 + 0.00014955252959987105im -2.4745122474312216 - 0.0014218201970612053im 2.54395032957965 - 0.0017617509218364057im -1.6317409308877564 + 0.014481262423566659im 0.6520209310217221 - 0.01723841568775108im 1.5728392695456896 - 0.05772879911474757im -1.8916301561345128 + 0.026699608625703128im 2.9913041452068194 - 0.14550318858855774im; 2.148100789014534 - 0.00013153659678511234im -0.8935700741779301 + 0.0im 2.148100789014534 + 0.00013153659678511234im -2.1684422425380014 - 0.001245966531666231im 2.229288793342719 - 0.0015462234484159487im -1.429916601644413 + 0.012695644758042742im 0.5713291902043836 - 0.01508776687705352im 1.378221696772371 - 0.050668432784474825im -1.6576326407911486 + 0.023109995650190698im; -1.8623722376447815 + 0.0010701128662712566im 1.8448999653498332 - 0.00011352066397035364im -0.6999932888534828 + 0.0im 1.8448999653498332 + 0.00011352066397035364im 

9×9 Matrix{ComplexF64}:
  -1.08715+0.0im             2.4513+0.000149553im  …     2.9913-0.145503im
    2.1481-0.000131537im   -0.89357+0.0im              -1.65763+0.02311im
  -1.86237+0.00107011im      1.8449-0.000113521im        1.1836-0.0436081im
   1.59997+0.00111517im     -1.5563+0.000894259im      0.409946-0.0107865im
 -0.824444-0.00733879im      1.2853+0.000899641im     -0.824444+0.00733879im
  0.248562+0.00648517im   -0.622619-0.00555317im   …   0.970643-0.000684114im
  0.405134+0.0153666im      0.16787+0.00433452im      -0.638092-0.000366698im
 -0.253648-0.00157232im    0.210516+0.00830623im       0.328896+2.3441e-5im
 0.0298474+0.000944072im   -0.01965+0.0020173im        0.461467+0.0im